Required knowledges
===================

* ``numpy`` (basic knowledge)
* ``Qt`` (basic knowledge)
* ``h5py`` (optionally)

Useful silx resources
=====================

* Getting start with the Hdf5 ([http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html](http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html))

Exercises summary
=================

1. Features provided by Hdf5TreeView
      * Learn how to create an ``Hdf5TreeView``
2. Create a HDF5 viewer
      * Learn how to use a dataset displayed by the tree
3. Create a tool to aggregate dataset images
      * Use multi-selection node of the tree
      * Use features of the mask
4. Create a phase contrast viewer
      * Learn how to add context menu to the tree nodes


Features provided by Hdf5TreeView
=================================

![HDF5 Tree](images/display.png)

In [1]:
from silx.gui import qt
from silx.gui import hdf5

%gui qt
app = qt.QApplication([])
tree = hdf5.Hdf5TreeView()
tree.setVisible(True)
app.exec_()

0

> Exercise:
>
> 1. Execute this script
> 2. Drag and drop an HDF5 file and play with it

> Exercise:
>
> 1. Use can use `exercices/ex1_display.py` as skeleton
> 2. Create an application to load HDF5 from command line
>       * Use [getting started with HDF5 widgets](http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html)

Solution
--------

In [2]:
!./working_examples/display.py nexus-20110325.h5

Gtk-Message: Failed to load module "canberra-gtk-module"


Create an HDF5 viewer
=====================

This exercise you how to use the `Hdf5TreeView` to browse and display datasets.
We provide a `DataViewer` widget to help you to display the data.

![HDF5 viewer](images/viewer.png)

DataViewer
----------

We provide a `DataViewer` widget, to display data using `Silx` plots.

In [3]:
from silx.gui import plot

class DataViewer(qt.QStackedWidget):
    """Widget to display any kind of data"""

    def __init__(self, parent=None):
        """Constructor"""
        super(DataViewer, self).__init__(parent)

        self.__plot1d = plot.Plot1D()
        self.__plot2d = plot.Plot2D()
        self.__text = qt.QLabel()
        self.__text.setAlignment(qt.Qt.AlignCenter)

        self.__index1d = self.addWidget(self.__plot1d)
        self.__index2d = self.addWidget(self.__plot2d)
        self.__indexText = self.addWidget(self.__text)
        self.setCurrentIndex(self.__indexText)

    def showAsString(self, data):
        """Display a data using text"""
        self.__text.setText(str(data))
        self.setCurrentIndex(self.__indexText)

    def show1d(self, data):
        """Display a data using silx Plot1D"""
        self.__plot1d.clear()
        self.__plot1d.addCurve(legend="data", x=range(len(data)), y=data)
        self.setCurrentIndex(self.__index1d)

    def show2d(self, data):
        """Display a data using silx Plot2D"""
        self.__plot2d.clear()
        self.__plot2d.addImage(legend="data", data=data)
        self.setCurrentIndex(self.__index2d)

    def show(self, data):
        """Display a data using the widget which fit the best"""
        isAtomic = len(data.shape) == 0
        isCurve = len(data.shape) == 1 \
                        and numpy.issubdtype(data.dtype, numpy.number)
        isImage = len(data.shape) == 2 \
                        and numpy.issubdtype(data.dtype, numpy.number)
        if isAtomic:
            self.showAsString(data)
        elif isCurve:
            self.show1d(data)
        elif isImage:
            self.show2d(data)
        else:
            self.showAsString(data)

Here is an example of use.

In [4]:
import numpy
viewer = DataViewer()
viewer.setVisible(True)

In [5]:
# To display an image
viewer.show(numpy.random.rand(100, 100))

In [6]:
# or a curve
viewer.show(numpy.random.rand(100))

In [7]:
# or a value
viewer.show(numpy.random.rand(1)[0])

Hdf5TreeView with DataViewer
----------------------------

Here is a code to display together an `Hdf5TreeView` and a `DataViewer`

In [8]:
window = qt.QSplitter()
tree = hdf5.Hdf5TreeView(window)
viewer = DataViewer(window)
window.addWidget(tree)
window.addWidget(viewer)
window.setStretchFactor(1, 1)
window.setVisible(True)

> Exercise (you can use `exercises/ex2_viewer.py` as skeleton)
>
> 1. Connect the tree to the viewer
>       * Use [getting started with HDF5 widgets](http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html)

Solution
--------

In [9]:
!./working_examples/viewer.py nexus-20110325.h5

Gtk-Message: Failed to load module "canberra-gtk-module"


Create an aggregation from diffraction acquisition
================================================

This exercise show how to configure and use the `Hdf5TreeView` with multi-selection. It will be used to compute an aggregation on images. The use case is an aggregation of diffraction acquisitions in order to create a better mask.

![HDF5 diffraction mask](images/diffraction_mask_log.png)

> Exercise (you can use `exercises/ex3_diffraction_mask.py` as skeleton)
>
> 1. Configure the tree as multi-selectable
>       * Use [`QAbstractItemView` documentation](http://doc.qt.io/qt-4.8/qabstractitemview.html#selectionMode-prop)
> 2. Aggregate selected datasets on `onTreeActivated`
> 3. Show the result in the viewer
> 4. With the GUI, use the mask tool to create a mask from aggregated images

Solution
--------

In [11]:
!./working_examples/diffraction_mask.py ID22_ma2909_Ti37Nb_450_72h_1.h5

Gtk-Message: Failed to load module "canberra-gtk-module"


Create an phase contrast viewer
===============================

This exercice show how to use the `Hdf5TreeView` context menu to a custom use. The use case is the phase contrast acquisition, in order to display better images from the raw data. To correct this images, we have to remove a background and apply a flat field. We can use the context menu to identify this dataset from an HDF5 file. The exercice provides few functions to help the computation.

![HDF5 phase contrast viewer](images/phase_contrast_corrected.png)

Provided functions
------------------

The computation of corrected images is done using this equation using `raw`, `flatfield`, and `background` information.

$$corrected = \frac{raw - background}{flatfield - background}$$

In [12]:
def computeCorrectedImage(raw):
    if FLATFIELD is None:
        raise RuntimeError("No flatfield defined")
    if BACKGROUND is None:
        raise RuntimeError("No background defined")

    numerator = raw.value - BACKGROUND.value
    denominator = FLATFIELD.value - BACKGROUND.value
    return numpy.array(numerator, dtype=numpy.float32) / denominator

def setBackground(dataset):
    """"Callback to define the background"""
    global BACKGROUND
    BACKGROUND = dataset

def setFlatField(dataset):
    """"Callback to define the flat field"""
    global FLATFIELD
    FLATFIELD = dataset

> Exercise (you can use `exercises/ex4_phase_contrast.py` as skeleton)
>
> 1. Register a callback function for the context menu of the tree
>       * Use [getting started with HDF5 widgets](http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html)
> 2. Create action to the menu to use the hovered dataset as backround of flatfield
>       * Use [getting started with HDF5 widgets](http://pythonhosted.org/silx/modules/gui/hdf5/getting_started.html)
> 3. Try to compute the corrected image when an image is selected in the tree and show it in the viewer

Solution
--------

In [13]:
!./working_examples/phase_contrast.py ID16B_diatomee.h5

Gtk-Message: Failed to load module "canberra-gtk-module"
